In [1]:
import pandas as pd
import numpy as np
import math
import sys  
sys.path.insert(0, '../src')
from dnns3 import *

In [ ]:
def get_previous_store_lower(i, dfs):
    """ Helper method used in prep_dataFrame.
        We assume df has a column called 'v0-l' and that if the 
        index is j then row j contains the data for the j^th
        instruction. We assume that df contains only the stores.
        
        We grab the lower 32 (least significant) bits of the
        first store which occurs prior to the i^th instruction
        and return this value.
    """
    idx = dfs[dfs.index < i].index.max()
    if (pd.isnull(idx)):
        return np.nan
    return dfs.loc[idx, 'v0-l']

def get_previous_store_upper(i, dfs):
    idx = dfs[dfs.index < i].index.max()
    if (pd.isnull(idx)):
        return np.nan
    return dfs.loc[idx, 'v0-u']

def get_previous_store_lower2(i, dfs):
    """ Helper method used in prep_dataFrame.
        We assume df has a column called 'v0-l' and that if the 
        index is j then row j contains the data for the j^th
        instruction. We assume that df contains only the stores.
        
        We grab the lower 32 (least significant) bits of the
        first store which occurs prior to the i^th instruction
        and return this value.
    """
    idx = dfs[dfs.index < i].index.max()
    idx = dfs[dfs.index < idx].index.max()
    if (pd.isnull(idx)):
        return np.nan
    return dfs.loc[idx, 'v0-l']

def get_previous_store_upper2(i, dfs):
    idx = dfs[dfs.index < i].index.max()
    idx = dfs[dfs.index < idx].index.max()
    if (pd.isnull(idx)):
        return np.nan
    return dfs.loc[idx, 'v0-u']

In [ ]:
def prepDataFrame(filename, k=4, read_stores=True):
	""" Read in the loads and stores from a trace and 
	"""

	# Read in the data and get it ready for computation
	df = pd.read_csv(filename)
	df = df.replace('NAN', np.nan)
	df['pc'] = df['pc'].astype('int64')
	df['effective_address'] = df['effective_address'].astype('int64')
	df['val0'] = df['val0'].astype('int64')
    
    # Bits: uu mu ml ll
    # uu := most significant group of 16 bits
    # mu := 2nd most significant group of 16 bits
    # ml := 3rd most significant group of 16 bits
    # ll := least significant group of 16 bits

	# Build a dataframe with the 64bit values split into four 16 bit values
	df1 = pd.DataFrame(index=df[df['type'] == 'l'].index) # only the loads
	df1['pc-ll'] = df['pc'] % math.pow(2, 15) # lower 16 bits
    df1['pc-ml'] = (df['pc'] - df1['pc-ll']) % math.pow(2, 31) 
	df1['ea-l'] = df['effective_address'] % math.pow(2, 15) # lower 16 bits
	df1['ea-u'] = df['effective_address'] - df1['ea-l'] # upper 48 bits
    
    
    
    #TODO
	df1['v0-l'] = df['val0'] % math.pow(2, 31)
	df1['v0-u'] = df['val0'] - df1['v0-l']

	# Scale all values in df1 into the interval [0,1]
	df1['pc-l'] = df1['pc-l']/df1['pc-l'].max()
	df1['ea-l'] = df1['ea-l']/df1['ea-l'].max()
	df1['ea-u'] = df1['ea-u']/df1['ea-u'].max()
	df1['v0-l'] = df1['v0-l']/df1['v0-l'].max()
	df1['v0-u'] = df1['v0-u']/df1['v0-u'].max()

	# Add columns for the previous k loads
	for i in range(1, k+1):
	    lname = 'v0l-' + str(i)
	    df1[lname] = df1['v0-l'].shift(i)
	    uname = 'v0u-' + str(i)
	    df1[uname] = df1['v0-u'].shift(i)

	# Add columns for the first 32 and last 32 bits of the previous 2 store values
	# Create the file name where the store columns must be read/written from/to.
	store_file = filename[:-4] + 'store_cols.csv'
	if (read_stores):
		# The columns for stores have already been computed for this file. 
		# Read the file and put the columns in df1
		store_columns = pd.read_csv(store_file, index_col=0)
		df1['s-1l'] = store_columns['s-1l'].astype('float64')
		df1['s-1u'] = store_columns['s-1u'].astype('float64')
		df1['s-2l'] = store_columns['s-2l'].astype('float64')
		df1['s-2u'] = store_columns['s-2u'].astype('float64')
	else:
		# The columns for the stroes have not been previously computed. Compute
		# these columns and write them to a file for later use
		dfs = pd.DataFrame(index=df[df['type'] == 's'].index)
		dfs['v0-l'] = (df['val0'] % math.pow(2, 31)).astype('float64')
		dfs['v0-u'] = (df['val0'] - dfs['v0-l']).astype('float64')
		dfs['v0-l'] = dfs['v0-l']/dfs['v0-l'].max()
		dfs['v0-u'] = dfs['v0-u']/dfs['v0-u'].max()
		df1['s-1l'] = df1.index.to_series().apply(lambda x : get_previous_store_lower(x, dfs))
		df1['s-1u'] = df1.index.to_series().apply(lambda x : get_previous_store_upper(x, dfs))
		df1['s-2l'] = df1.index.to_series().apply(lambda x : get_previous_store_lower2(x, dfs))
		df1['s-2u'] = df1.index.to_series().apply(lambda x : get_previous_store_upper2(x, dfs))
		store_columns = pd.DataFrame(index=df1.index)
		store_columns['s-1l'] = df1['s-1l']
		store_columns['s-1u'] = df1['s-1u']
		store_columns['s-2l'] = df1['s-2l']
		store_columns['s-2u'] = df1['s-2u']
		store_columns.to_csv(store_file)

	# Reorder the columns so the 'outputs' are the last 2 columns
	df1 = df1[["pc-l", "ea-l", "ea-u", "v0l-1", "v0u-1", "v0l-2", "v0u-2", "v0l-3", \
	           "v0u-3", "v0l-4", "v0u-4", "s-1l", "s-1u", "s-2l", "s-2u", "v0-l", "v0-u"]]

	# Drop all rows with nans
	return df1.dropna()

In [3]:
filename = "../data/1M-LdSt.csv"
df = prepDataFrame(filename)
num_training_examples = int(0.8*len(df))
X_tr, X_te, y_tr, y_te = test_train_split(df, num_training_examples)

In [3]:
widths = [100] #[100, 90, 80, 70, 60, 50]#[40,30,20,10,5]
rates  = [0.00075] #[0.002, 0.001, 0.0005]#[0.001, 0.0001, 0.00001]
epochs = [100] #[100]
exp_data = pd.DataFrame(columns=['epochs', 'width', 'rate', 'error'])

best_model = build_model()
best_error = 10000

row = 0
for e in epochs:
    for w in widths:
        for r in rates:
            # do 5 experiments for each hyperparameter combo
            for i in range(5):
                print("---e = " + str(e) + " w = " + str(w) + ' r = ' + str(r) + ' i = ' + str(i) + "---")
                # build, train, and test model. then compute the error
                model = build_model(do1=w, do2=w, do3=w, do4=w, do5=w, l_rate=r)
                model.fit(X_tr, y_tr, epochs=e, verbose=False)
                predictions = model.predict(X_te)
                rmse = get_rmse(predictions, y_te, y_te.shape[0])

                # record the network paramaters and error
                exp_data.loc[row] = [e, w, r, rmse]
                row += 1

                print("rmse = " + str(rmse))
                # see if this model is the best yet, if so save it
                if rmse < best_error:
                    best_error = rmse
                    best_model = model

# record the experimental data
exp_data.to_csv(filename[:-4] + '-experiments.csv',index=False)
# Save the best model
best_model.save(filename[:-4] + '-best-model')


---e = 100 w = 100 r = 0.00075 i = 0---
rmse = 0.0012421019647526953
---e = 100 w = 100 r = 0.00075 i = 1---
rmse = 0.0012698088988954946
---e = 100 w = 100 r = 0.00075 i = 2---
rmse = 0.0012797240004229307
---e = 100 w = 100 r = 0.00075 i = 3---
rmse = 0.0012935658062535635
---e = 100 w = 100 r = 0.00075 i = 4---
rmse = 0.0012672786043439808
INFO:tensorflow:Assets written to: 1M-LdSt-best-model/assets


In [ ]:
# To load a previously saved model:
# model = keras.models.load_model('path/to/location')

In [ ]:
# break everyting up into 16 bits, if that doesnt work then add more stores.